In [1]:
import numpy as np

s = np.array([
    [[1, 0],
     [0, 0]],
    [[0, 0],
     [0, 0]],
    [[1, 1],
     [1, 0]],
    [[0, 0],
     [1, 1]],
])

In [4]:
from functools import partial
import pyomo.environ as pyo


def Model(s):
    m = pyo.ConcreteModel()

    # init decision variable, for convenience we convert the pyomo `Var`
    # type to a `numpy` array
    x = []
    for n in range(s.shape[0]):
        for i in range(s.shape[1]):
            for j in range(s.shape[2]):
                setattr(m, f'x{n}_{i},{j}', pyo.Var(domain=pyo.Binary))
                x.append(getattr(m, f'x{n}_{i},{j}'))
    x = np.array(x).reshape(s.shape)

    # objective
    def objective(model):
        return -x.sum()
    m.obj = pyo.Objective(rule=objective)

    # constraint: use only once
    m.constraint_use_once = pyo.ConstraintList()
    for c in x.sum(axis=(1, 2)):
        m.constraint_use_once.add(c <= 1)
    
    # constraint: satisfy criteria
    m.constraint_criteria = pyo.ConstraintList()
    for i in np.arange(s.shape[1]):
        for j in np.arange(s.shape[2]):
            m.constraint_criteria.add(
                (x[:,i,j] * s[:,i,j]).sum() == x[:,i,j].sum()
            )
    
    # constraint: one solution per box
    m.constraint_one_solution = pyo.ConstraintList()
    for i in np.arange(s.shape[1]):
        for j in np.arange(s.shape[2]):
            m.constraint_one_solution.add(
                x[:,i,j].sum() <= 1
            )

    return m, x


model, x = Model(s)
opt = pyo.SolverFactory('cbc')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -3.0, 'Upper bound': -3.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 6, 'Number of binary variables': 16, 'Number of integer variables': 16, 'Number of nonzeros': 6, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.018182992935180664}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [5]:
model.pprint()

3 Set Declarations
    constraint_criteria_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    constraint_one_solution_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    constraint_use_once_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}

16 Var Declarations
    x0_0,0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    x0_0,1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary
    x0_1,0 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : Fals

In [6]:
x_star = np.array([pyo.value(sv) for sv in x.ravel()]).reshape(x.shape)
x_star

array([[[1., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [1., 0.]],

       [[0., 0.],
        [0., 1.]]])

In [7]:
s = np.random.choice([0, 1], p=[.9, .1], size=(10, 3, 3))
model, x = Model(s)
opt = pyo.SolverFactory('cbc')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -8.0, 'Upper bound': -8.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 14, 'Number of binary variables': 90, 'Number of integer variables': 90, 'Number of nonzeros': 14, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.020322084426879883}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
x_star = np.array([pyo.value(sv) for sv in x.ravel()]).reshape(x.shape)
x_star

array([[[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.]],

       [[0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 1.]]])

# Immaculate Grid

In [9]:
import pandas as pd


# Get rosters

base_urls = [
    'https://www.baseball-reference.com/teams/{TEAM}/bat.shtml',
    'https://www.baseball-reference.com/teams/{TEAM}/pitch.shtml',
]

teams = [
    'PHI',
    'SEA',
    'STL',
    'HOU',
    'COL',
    'KC',
]

rosters = []
for team in teams:
    for url in base_urls:
        rosters.append(
            pd.read_html(url.format(TEAM=team))[0].assign(team=team)
        )

In [10]:
rosters = pd.concat(rosters)
rosters

,Rk,Name,S,C,F,Yrs,From,To,ASG,G,...,SHO,SV,IP,ER,IBB,HBP,BK,WP,BF,ERA+
0,1,Ed Abbaticchio,S,-,-,2,1897,1898,0,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Fred Abbott,-,-,F,1,1905,1905,0,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Kyle Abbott,-,-,-,2,1992,1995,0,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Paul Abbott,-,-,F,1,2004,2004,0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Bobby Abreu,-,-,-,9,1998,2006,2,1353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,516,Curt Young,-,-,-,1,1992,1992,0,10,...,0,0,24.1,14,1,0,0,0,107,79
526,517,Chris Zachary,-,-,-,1,1969,1969,0,8,...,0,0,18.1,16,0,0,0,4,87,48
527,518,Angel Zerpa,S,C,F,3,2021,2023,0,19,...,0,0,58.2,25,0,4,0,2,246,116
528,519,Kyle Zimmer,S,C,F,3,2019,2021,0,83,...,0,2,95.1,55,1,1,0,13,416,90


In [15]:
rosters.to_csv('rosters.csv.gz', index=False)

In [11]:
rosters['active'] = 1
rosters = rosters.pivot_table(index='Name', columns='team', values='active').fillna(0)
rosters

team,COL,HOU,KC,PHI,SEA,STL
Name,,,,,,
A.J. Burnett,0.0,0.0,0.0,1.0,0.0,0.0
A.J. Ellis,0.0,0.0,0.0,1.0,0.0,0.0
A.J. Hinch,0.0,0.0,1.0,1.0,0.0,0.0
A.J. Pierzynski,0.0,0.0,0.0,0.0,0.0,1.0
A.J. Sager,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
Zak Shinall,0.0,0.0,0.0,0.0,1.0,0.0
Zeke Wilson,0.0,0.0,0.0,0.0,0.0,1.0
Zinn Beck,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
import numpy as np


gridx = ['PHI', 'SEA', 'STL']
gridy = ['HOU', 'COL', 'KC']

rosters = rosters.sample(frac=1)
s = np.zeros((len(rosters), 3, 3))

for i, cond1 in enumerate(gridy):
    for j, cond2 in enumerate(gridx):
        s[np.arange(len(s)),i,j] = (rosters[cond1] + rosters[cond2] == 2).astype(int)
print(s.shape)
s

(6601, 3, 3)


array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [13]:
model, x = Model(s)
opt = pyo.SolverFactory('cbc')
opt.solve(model)
x_star = np.array([pyo.value(sv) for sv in x.ravel()]).reshape(x.shape)
solution = pd.DataFrame(columns=gridx, index=gridy)
for ix, i, j in np.argwhere(x_star == 1):
    solution.iloc[i,j] = rosters.index[ix]
solution

,PHI,SEA,STL
HOU,Eric Bruntlett,Raul Chavez,Lance Berkman
COL,Kane Davis,Greg Colbrunn,Chris Richard
KC,Mark Ryal,Mike Montgomery,Vada Pinson
